In [11]:
from syft import PhiTensor as PT

In [12]:
from syft.core.adp.data_subject_list import DataSubject
import numpy as np

ishan = np.array([DataSubject(["Ishan"])])

In [13]:
from syft import lazyrepeatarray as lra

upper_bound = lra(data=5, shape=(3,3))
lower_bound = lra(data=-5, shape=(3,3))

reference_data = np.random.randint(-5, 5, (3,3))

In [14]:
reference_tensor = PT(
    child=np.array(reference_data),
    data_subject=ishan,
    max_vals=upper_bound,
    min_vals=lower_bound,
)
gamma_tensor = reference_tensor.gamma

aux_tensor = gamma_tensor == reference_data
result = aux_tensor.any()
assert result.child
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

result = (gamma_tensor == reference_data).any(axis=0)
assert result.shape == (reference_data.shape[0],)
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

result = (gamma_tensor == reference_data).any(keepdims=True)
assert result.shape == (1, 1)
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

result = (gamma_tensor == reference_data).any(keepdims=True, axis=0)
assert result.shape == (1, reference_tensor.shape[0])
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

condition = list(
    np.random.choice(a=[False, True], size=(reference_data.shape[0] - 1))
)
condition.append(
    True
)  # If condition = [False, False, False ... False], this test will fail
result = (gamma_tensor == reference_data).any(where=condition)
assert result.child
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
# print(state)
# print(result.func(state))

assert result.func(state).shape == result.child.shape
assert (result.func(state) == result.child).all()  # TODO 0.7: debug this

In [15]:
tensor = PT(
    child=reference_data,
    data_subject=ishan,
    min_vals=lower_bound,
    max_vals=upper_bound,
)
gamma_tensor = tensor.gamma
output = gamma_tensor // 5
assert (output.child == (reference_data // 5)).all()
assert list(output.sources.keys()) == [tensor.id]
state = {}
for key in output.sources:
    state[key] = output.sources[key].child
assert (output.func(state) == output.child).all()

tensor2 = PT(
    child=reference_data + 1,
    data_subject=ishan,
    min_vals=lower_bound + 1,
    max_vals=upper_bound + 1,
)

output = gamma_tensor // tensor2
assert (output.child == (reference_data // (reference_data + 1))).all()
assert list(output.sources.keys()) == [tensor.id, tensor2.id]
state = {}
for key in output.sources:
    state[key] = output.sources[key].child
print(output.child)
print(output.func(state))

[[0 1 0]
 [0 0 2]
 [0 0 0]]
[[ 0  1 -2]
 [ 0  0  2]
 [ 0  0  0]]


/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/tensor/autodp/gamma_tensor.py:3367: RuntimeWarning: divide by zero encountered in floor_divide
  sources.update(other.sources)
/tmp/ipykernel_26149/1547835576.py:24: RuntimeWarning: divide by zero encountered in floor_divide
  assert (output.child == (reference_data // (reference_data + 1))).all()


In [16]:
assert (output.func(state) == output.child).all()  # TODO 0.7: fix this one

AssertionError: 